# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [29]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [30]:
# 2) Quantidade de dados faltantes por variável.
pd.DataFrame(sinasc.isna().sum(), columns=['Quantidade de dados faltantes'])

,Quantidade de dados faltantes
ORIGEM,0
CODESTAB,115
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
...,...
munResUf,0
munResLat,1
munResLon,1
munResAlt,1


In [31]:
# 3) 'sinasc_selecao' - DF com as apenas as variáveis de interesse.

sinasc_selecao = sinasc.loc[:, ['LOCNASC',
                                'IDADEMAE',
                                'ESTCIVMAE',
                                'ESCMAE',
                                'QTDFILVIVO',
                                'GESTACAO',
                                'GRAVIDEZ',
                                'CONSULTAS',
                                'APGAR5']
                           ]
sinasc_selecao

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [32]:
# 3) Quantidade de dados faltantes por variável da seleção de interesse.

pd.DataFrame(sinasc_selecao.isna().sum(), columns=['Quantidade de dados faltantes'])

,Quantidade de dados faltantes
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,317
ESCMAE,312
QTDFILVIVO,1573
GESTACAO,1232
GRAVIDEZ,79
CONSULTAS,0
APGAR5,103


In [33]:
# 4) Muda o DF 'sinasc_seleção' retirando as linhas com dados faltantes da variável 'APGAR5'.

sinasc_selecao.dropna(subset=['APGAR5'], inplace=True)

In [34]:
# 4) Verifica após mudança acima no DF 'sinasc_selecao' a quantidade de dados faltantes por variável de interesse.

pd.DataFrame(sinasc_selecao.isna().sum(), columns=['Nova quantidade de dados faltantes'])

,Nova quantidade de dados faltantes
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,315
ESCMAE,310
QTDFILVIVO,1566
GESTACAO,1216
GRAVIDEZ,76
CONSULTAS,0
APGAR5,0


In [35]:
# 5) Substitui os valores faltantes de 'ESTCIVMAE' e 'CONSULTAS' por "9".

sinasc_selecao['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_selecao['CONSULTAS'].fillna(9, inplace=True)

In [36]:
# 6) Substitui os valores faltantes de 'QTDFILVIVO' por zero.

sinasc_selecao['QTDFILVIVO'].fillna(0, inplace=True)

In [37]:
# 7) Substitui os valores faltantes de 'ESCMAE', 'GESTACAO', 'GRAVIDEZ' por "9".

sinasc_selecao['ESCMAE'].fillna(9, inplace=True)
sinasc_selecao['GESTACAO'].fillna(9, inplace=True)
sinasc_selecao['GRAVIDEZ'].fillna(9, inplace=True)

7) Como os valores de 'ESCMAE', 'GESTACAO', 'GRAVIDEZ' sendo categóricos, achei mais prudente substituir os valores faltantes por "9 - ignorado".
Achei melhor analisar os dados considerando que essa informação não foi dada, ao invés de substituir e acabar dando um viés ao resultado.

In [38]:
# 8) Cria 'APGAR5_CLASS' e mostra sua frequência

sinasc_selecao.loc[ (sinasc_selecao['APGAR5'] >= 0) & (sinasc_selecao['APGAR5'] <= 3), 'APGAR5_CLASS' ] = 'asfixia severa'
sinasc_selecao.loc[ (sinasc_selecao['APGAR5'] >= 4) & (sinasc_selecao['APGAR5'] <= 5), 'APGAR5_CLASS' ] = 'asfixia moderada' 
sinasc_selecao.loc[ (sinasc_selecao['APGAR5'] >= 6) & (sinasc_selecao['APGAR5'] <= 7), 'APGAR5_CLASS' ] = 'asfixia leve' 
sinasc_selecao.loc[ (sinasc_selecao['APGAR5'] >= 8) & (sinasc_selecao['APGAR5'] <= 10), 'APGAR5_CLASS' ] = 'normal'

sinasc_selecao['APGAR5_CLASS'].value_counts()

APGAR5_CLASS
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [43]:
# 9) Renomeia colunas.

sinasc_selecao.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
                          'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_class']
sinasc_selecao.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_class'],
      dtype='object')